<a href="https://colab.research.google.com/github/ShivikaPrasanna/brain_cancer_gene_expression/blob/main/Brain_cancer_with_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
!sudo pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

In [101]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import KFold, StratifiedKFold

import imblearn
from imblearn.over_sampling import SMOTE

In [102]:
# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Brain_GSE50161.csv")
data.groupby('type')['type'].count()

type
ependymoma               46
glioblastoma             34
medulloblastoma          22
normal                   13
pilocytic_astrocytoma    15
Name: type, dtype: int64

In [103]:
df_class_pilocytic_astrocytoma = data[data['type'] == 'pilocytic_astrocytoma']
df_class_normal = data[data['type'] == 'normal']
data = data.append(df_class_pilocytic_astrocytoma)
data = data.append(df_class_normal)
data.groupby('type')['type'].count()

<ipython-input-103-5b1a81a48be2>:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-103-5b1a81a48be2>:4: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



type
ependymoma               46
glioblastoma             34
medulloblastoma          22
normal                   26
pilocytic_astrocytoma    30
Name: type, dtype: int64

In [104]:
cancer_type_dict = {'ependymoma': 0, 'glioblastoma': 1, 'medulloblastoma': 2, 'pilocytic_astrocytoma': 3, 'normal': 4}
data['mapped_type'] = data['type'].replace(cancer_type_dict)
data.drop('type', axis=1, inplace=True)
data['samples'].groupby(data['mapped_type']).count()
labels = data.pop('mapped_type')

In [105]:
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(data, labels)

In [118]:
print(len(X))
print(set(labels))
data = X.to_numpy().astype(np.float32)
labels = y.to_numpy().flatten().astype(np.float32)

230
{0, 1, 2, 3, 4}


In [119]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)

# # Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [120]:
# Define the model architecture
model = Sequential()
model.add(Dense(8, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [121]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min', restore_best_weights=True)
mcp_save = tf.keras.callbacks.ModelCheckpoint('my-model-{accuracy:.2f}.hdf5', save_best_only=True, monitor='accuracy', mode='max', verbose=1)
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1, mode='min')

In [122]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 8)                 437416    
                                                                 
 dense_31 (Dense)            (None, 4)                 36        
                                                                 
 dense_32 (Dense)            (None, 4)                 20        
                                                                 
 dense_33 (Dense)            (None, 1)                 5         
                                                                 
Total params: 437,477
Trainable params: 437,477
Non-trainable params: 0
_________________________________________________________________


In [123]:
# Train the model
history = model.fit(X_train, y_train,  epochs=50, batch_size=32)

Epoch 1/50
7/7 [==============================] - 2s 23ms/step - loss: -12.6172 - accuracy: 0.2367
Epoch 2/50
7/7 [==============================] - 0s 24ms/step - loss: -66.9865 - accuracy: 0.2850
Epoch 3/50
7/7 [==============================] - 0s 21ms/step - loss: -151.5408 - accuracy: 0.2995
Epoch 4/50
7/7 [==============================] - 0s 21ms/step - loss: -259.7625 - accuracy: 0.2850
Epoch 5/50
7/7 [==============================] - 0s 24ms/step - loss: -378.0596 - accuracy: 0.2947
Epoch 6/50
7/7 [==============================] - 0s 21ms/step - loss: -528.4092 - accuracy: 0.2995
Epoch 7/50
7/7 [==============================] - 0s 21ms/step - loss: -691.0190 - accuracy: 0.2947
Epoch 8/50
7/7 [==============================] - 0s 23ms/step - loss: -905.1919 - accuracy: 0.3043
Epoch 9/50
7/7 [==============================] - 0s 21ms/step - loss: -1122.7581 - accuracy: 0.2947
Epoch 10/50
7/7 [==============================] - 0s 20ms/step - loss: -1389.1934 - accuracy: 0.2995

In [124]:
import json
with open('history.json', 'w') as f:
    json.dump(history.history, f)

In [125]:
import codecs
obj_text = codecs.open('history.json', 'r', encoding='utf-8').read()
hist = json.loads(obj_text)

In [126]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter( y=hist['loss'], name="loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=hist['accuracy'], name="train accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss/Accuracy of Neural Network Model"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

In [134]:
score, acc = model.evaluate(X_test, y_test, batch_size=32, verbose=2)
print("Accuracy: ", acc)

from sklearn.metrics import f1_score, precision_score, recall_score
y_pred = model.predict(X_test).astype(int)

# Print f1, precision, and recall scores
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="micro"))

1/1 - 0s - loss: -1.1936e+05 - accuracy: 0.2174 - 53ms/epoch - 53ms/step
Accuracy:  0.21739129722118378
1/1 [==============================] - 0s 40ms/step
0.17105263157894737
0.32
0.21739130434782608


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

